# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
# Determining the number of views and clicks per group
control_views = df[(df['group'] == 'control') & (df['action'] == 'view')].shape[0]
control_clicks = df[(df['group'] == 'control') & (df['action'] == 'click')].shape[0]
experiment_views = df[(df['group'] == 'experiment') & (df['action'] == 'view')].shape[0]
experiment_clicks = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]

# Determining the click-through rates (CTR)
control_ctr = control_clicks / control_views
experiment_ctr = experiment_clicks / experiment_views

# Conducting a chi-squared test
contingency_table = [[control_clicks, control_views - control_clicks],
                     [experiment_clicks, experiment_views - experiment_clicks]]

chi2, p_val, _, _ = stats.chi2_contingency(contingency_table)
p_val

0.00957168049704227

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = control_ctr * experiment_views
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:
#Your code here
# Calculate standard deviation of the number of clicks
std_dev = (experiment_views * control_ctr * (1 - control_ctr)) ** 0.5

# Calculate the z-score
z_score = (experiment_clicks - expected_experiment_clicks) / std_dev
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [6]:
#Your code here
# Calculate the p-value using the z-score
p_value_from_z = stats.norm.sf(abs(z_score)) * 2  # Two-tailed test
p_value_from_z

0.00024973056013902396

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [7]:
"""
The p_value is much far below the significance level.
This indicates that there is sufficient evidence to suggest that the experimental homepage led to more clicks than the control.
"""

'\nThe p_value is much far below the significance level.\nThis indicates that there is sufficient evidence to suggest that the experimental homepage led to more clicks than the control.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.